# Imports


In [1]:
import random
import gradio as gr


# Flower



In [2]:
class Flower:
    def __init__(self, sizeCenter , colorCenter , colorPetals , colorStem , numPetals): 
        self.sizeCenter = sizeCenter
        self.colorCenter = colorCenter
        self.colorPetals = colorPetals
        self.colorStem = colorStem
        self.numPetals = numPetals
        
    def display(self):
        print(f"------------- Flower Details: -------------")
        print(f"Center Size: {self.sizeCenter}")
        print(f"Center Color: {self.colorCenter}")
        print(f"Petals Color: {self.colorPetals}")
        print(f"Stem Color: {self.colorStem}")
        print(f"Number of Petals: {self.numPetals}")



# Population


In [3]:
# Assume sizeCenter is 8 bits 
# Assume numPetals is 8 bits
# we may change the colorCenter, colorPetals, colorStem to be 24 bits each (8 bits for R, 8 bits for G, 8 bits for B)

def createPopulation():
    population = []  
    for i in range(8):
        sizeCenter = random.randint(0, 255)
        colorCenter = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorPetals = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorStem = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        numPetals = random.randint(0, 7)
        flower = Flower(sizeCenter, colorCenter, colorPetals, colorStem, numPetals)
        population.append(flower)
    return population

population = createPopulation()
population[0].display()
population[1].display()
population[2].display()
population[3].display()
population[4].display()
population[5].display()
population[6].display()
population[7].display()

------------- Flower Details: -------------
Center Size: 39
Center Color: (201, 248, 207)
Petals Color: (238, 103, 85)
Stem Color: (194, 165, 134)
Number of Petals: 3
------------- Flower Details: -------------
Center Size: 95
Center Color: (7, 253, 81)
Petals Color: (160, 94, 183)
Stem Color: (58, 176, 36)
Number of Petals: 4
------------- Flower Details: -------------
Center Size: 154
Center Color: (212, 194, 97)
Petals Color: (133, 136, 71)
Stem Color: (42, 209, 10)
Number of Petals: 5
------------- Flower Details: -------------
Center Size: 214
Center Color: (53, 101, 114)
Petals Color: (35, 57, 75)
Stem Color: (106, 186, 87)
Number of Petals: 7
------------- Flower Details: -------------
Center Size: 55
Center Color: (215, 52, 66)
Petals Color: (64, 150, 56)
Stem Color: (8, 165, 125)
Number of Petals: 2
------------- Flower Details: -------------
Center Size: 144
Center Color: (243, 59, 136)
Petals Color: (3, 139, 233)
Stem Color: (213, 178, 120)
Number of Petals: 0
------------- 

In [4]:
def convertDecimalToBinary(n):
    return bin(n).replace("0b", "").zfill(8)

def convertDecimalToBinary3(n):
    return bin(n).replace("0b", "").zfill(3)

print(convertDecimalToBinary(7))


00000111


In [59]:
def displayBinaryRepresentation(flower):
    sizeCenterBinary = convertDecimalToBinary(flower.sizeCenter)
    colorCenterBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorCenter])
    colorPetalsBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorPetals])
    colorStemBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorStem])
    numPetalsBinary = convertDecimalToBinary3(flower.numPetals)
    
    chromosome = sizeCenterBinary + colorCenterBinary + colorPetalsBinary + colorStemBinary + numPetalsBinary
    return chromosome

def Chromosomes(population):
    result = []
    for i, flower in enumerate(population):
        result.append(displayBinaryRepresentation(flower))
    return result


for flower in population:
    flower.display()
    print("Binary Representation: ", displayBinaryRepresentation(flower))
    print()


Chromosomes = Chromosomes(population)
for i, chromosome in enumerate(Chromosomes):
    print(f"Flower {i+1} Chromosome: {chromosome}")     

------------- Flower Details: -------------
Center Size: 155
Center Color: (230, 166, 53)
Petals Color: (48, 26, 173)
Stem Color: (29, 53, 98)
Number of Petals: 0
Binary Representation:  10011011111001101010011000110101001100000001101010101101000111010011010101100010000

------------- Flower Details: -------------
Center Size: 101
Center Color: (66, 35, 96)
Petals Color: (253, 241, 119)
Stem Color: (252, 199, 177)
Number of Petals: 0
Binary Representation:  01100101010000100010001101100000111111011111000101110111111111001100011110110001000

------------- Flower Details: -------------
Center Size: 47
Center Color: (143, 115, 254)
Petals Color: (4, 111, 18)
Stem Color: (189, 240, 167)
Number of Petals: 2
Binary Representation:  00101111100011110111001111111110000001000110111100010010101111011111000010100111010

------------- Flower Details: -------------
Center Size: 58
Center Color: (127, 236, 211)
Petals Color: (251, 35, 205)
Stem Color: (140, 242, 19)
Number of Petals: 0
Binary Repres

In [6]:
def convertBinaryToDecimal(chromosome):
    decimal = 0
    for i in range(len(chromosome)):
        decimal += int(chromosome[i]) * (2 ** (len(chromosome) - 1 - i))
    return decimal
   

In [7]:
print(convertBinaryToDecimal("00110001"))

49


In [8]:
def repair(min_value, max_value,x_value):
    return max(min_value, min(max_value, x_value))

# min 0
# max 1000

# num > 1000 2000
# num < 0  -1000

In [9]:
# Create population  
# Create Flower 
# Selection 
# Crossover
# Mutation
# Fitness Function
# Main 
# GUI 

# Selection


# Crossover


In [11]:
def crossover( flower1, flower2):
    crossPos = random.randint(1, len(flower1)-1)
    # print(crossPos)

    # Perform crossover
    offspring1 = flower1[:crossPos] + flower2[crossPos:]
    offspring2 = flower2[:crossPos] + flower1[crossPos:]
    return offspring1, offspring2

flower3 = "011100000101"
flower4 = "100101101100"
print(crossover(flower3,flower4))


('010101101100', '101100000101')


# Mutation

In [60]:
def mutation(population):
    chromosome = displayBinaryRepresentation(population[0])
    print(f"Chromosome Length: {len(chromosome)} bits")
    print(f"Population Size: {len(population)} flowers")
    total_bits = len(chromosome) * len(population)
    num_mutations = int(total_bits * 0.05)  # 5% of the total bits
    print(f"Total bits: {total_bits}, Number of mutations to perform: {num_mutations}\n")  

    for i in range(num_mutations):
        flower_index = random.randint(0, len(population) - 1)
        print(f"Mutation {i + 1}: Flower {flower_index + 1}")
        flower = population[flower_index]
        chromosome = displayBinaryRepresentation(flower)
        
        mutation_pos = random.randint(0, len(chromosome) - 1)
        if chromosome[mutation_pos] == '0':
            mutated_bit = '1'
        else:
            mutated_bit = '0'
        new_chromosome = chromosome[:mutation_pos] + mutated_bit + chromosome[mutation_pos + 1:]

        print(f"Flower {flower_index + 1} mutated at position {mutation_pos}:")
        print(f"Old Chromosome: {chromosome}")
        print(f"New Chromosome: {new_chromosome}\n")

        # ------------------- My code ends here this is copilot suggestion -------------------

        # Update flower attributes based on new chromosome
        flower.sizeCenter = convertBinaryToDecimal(new_chromosome[0:8])
        flower.colorCenter = (
            convertBinaryToDecimal(new_chromosome[8:16]),
            convertBinaryToDecimal(new_chromosome[16:24]),
            convertBinaryToDecimal(new_chromosome[24:32])
        )
        flower.colorPetals = (
            convertBinaryToDecimal(new_chromosome[32:40]),
            convertBinaryToDecimal(new_chromosome[40:48]),
            convertBinaryToDecimal(new_chromosome[48:56])
        )
        flower.colorStem = (
            convertBinaryToDecimal(new_chromosome[56:64]),
            convertBinaryToDecimal(new_chromosome[64:72]),
            convertBinaryToDecimal(new_chromosome[72:80])
        )
        flower.numPetals = repair(0, 7, convertBinaryToDecimal(new_chromosome[80:83]))
mutation(population)

Chromosome Length: 83 bits
Population Size: 8 flowers
Total bits: 664, Number of mutations to perform: 33

Mutation 1: Flower 4
Flower 4 mutated at position 72:
Old Chromosome: 00111010011111111110110011010011111110110010001111001101100011001111001000010011000
New Chromosome: 00111010011111111110110011010011111110110010001111001101100011001111001010010011000

Mutation 2: Flower 4
Flower 4 mutated at position 20:
Old Chromosome: 00111010011111111110110011010011111110110010001111001101100011001111001010010011000
New Chromosome: 00111010011111111110010011010011111110110010001111001101100011001111001010010011000

Mutation 3: Flower 2
Flower 2 mutated at position 68:
Old Chromosome: 01100101010000100010001101100000111111011111000101110111111111001100011110110001000
New Chromosome: 01100101010000100010001101100000111111011111000101110111111111001100111110110001000

Mutation 4: Flower 5
Flower 5 mutated at position 68:
Old Chromosome: 1101000011100100001001111100111100100000101000101011110000

# GUI

In [ ]:
def rgb_str(rgb):
    return f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})"


def flower_frame_html(index, flower: Flower):
    return f"""
    <div class="flower-frame">
      <canvas id="flowerCanvas{index}" width="200" height="200"></canvas>
      <div id="timer{index}" class="timer">Timer: 0</div>
    </div>
    <script>
      (function() {{
        const canvas = document.getElementById("flowerCanvas{index}");
        const ctx = canvas.getContext("2d");
        const timerDisplay = document.getElementById("timer{index}");

        const centerSize = {flower.sizeCenter / 20:.2f};
        const centerColor = "{rgb_str(flower.colorCenter)}";
        const petalColor = "{rgb_str(flower.colorPetals)}";
        const stemColor = "{rgb_str(flower.colorStem)}";
        const numPetals = {flower.numPetals};

        function drawFlower() {{
          const centerX = 100, centerY = 100;
          const petalRadius = 15;
          const flowerRadius = 40;

          ctx.beginPath();
          ctx.strokeStyle = stemColor;
          ctx.lineWidth = 6;
          ctx.moveTo(100, 200);
          ctx.lineTo(100, 130);
          ctx.stroke();

          ctx.fillStyle = petalColor;
          for (let i = 0; i < numPetals; i++) {{
            const angle = (2 * Math.PI / numPetals) * i;
            const x = centerX + flowerRadius * Math.cos(angle);
            const y = centerY + flowerRadius * Math.sin(angle);
            ctx.beginPath();
            ctx.arc(x, y, petalRadius, 0, 2 * Math.PI);
            ctx.fill();
          }}

          ctx.beginPath();
          ctx.fillStyle = centerColor;
          ctx.arc(centerX, centerY, centerSize, 0, 2 * Math.PI);
          ctx.fill();
        }}

        drawFlower();

        let time = 0;
        let timerInterval = null;

        function startTimer() {{
          if (!timerInterval) {{
            timerInterval = setInterval(() => {{
              time += 1;
              timerDisplay.textContent = "Timer: " + time;
              window.flowerTimers[{index}] = time;
            }}, 100);
          }}
        }}

        function stopTimer() {{
          clearInterval(timerInterval);
          timerInterval = null;
        }}

        canvas.addEventListener("mouseenter", startTimer);
        canvas.addEventListener("mouseleave", stopTimer);
      }})();
    </script>
    """


def build_html(population):
    html = """
    <!doctype html>
    <html>
    <head>
      <meta charset="utf-8" />
      <style>
        body {
          font-family: Arial, sans-serif;
          display: flex;
          flex-wrap: wrap;
          justify-content: center;
          align-items: center;
          gap: 20px;
          padding: 20px;
          margin: 0;
        }
        .flower-frame {
          display: flex;
          flex-direction: column;
          align-items: center;
          justify-content: center;
          border: 2px solid #4CAF50;
          border-radius: 10px;
          background: #f9f9f9;
          width: 220px;
          height: 260px;
          box-shadow: 0 0 5px rgba(0,0,0,0.1);
          transition: box-shadow 0.3s ease;
        }
        .flower-frame:hover {
          box-shadow: 0 0 15px rgba(76, 175, 80, 0.6);
        }
        canvas {
          border-radius: 10px;
          background: #f9f9f9;
          cursor: pointer;
        }
        .timer {
          margin-top: 8px;
          font-size: 16px;
          font-weight: bold;
          color: #333;
        }
      </style>
    </head>
    <body>
    <script>window.flowerTimers = new Array(8).fill(0);</script>
    """
    for i, f in enumerate(population):
        html += flower_frame_html(i, f)
    html += "</body></html>"
    return html


# ---- Python-side logic ----
def get_best_three(timers):
    timers = [int(t) for t in timers]
    sorted_indices = sorted(range(len(timers)), key=lambda i: timers[i], reverse=True)
    top4 = sorted_indices[:4]

    result = "🏅 **Top 4 Flowers:**\n"
    medals = ["🥇", "🥈", "🥉", "🏅"]
    for rank, idx in enumerate(top4):
        result += f"{medals[rank]} Flower {idx+1} — Timer: {timers[idx]}\n"

    return result


# ---- Gradio UI ----
with gr.Blocks(theme="default") as demo:
    gr.Markdown("## 🌸 Hover over each flower to increase its timer")
    iframe_html = f'<iframe id="flowerFrame" srcdoc="{build_html(population).replace(chr(34), "&quot;")}" style="width:100%; height:850px; border:none;"></iframe>'
    gr.HTML(value=iframe_html)

    get_timers_btn = gr.Button("Show Top 3 Hovered Flowers")
    result_display = gr.Markdown("")

    get_timers_btn.click(
        fn=lambda timers: get_best_three(timers),
        js="window.flowerTimers",
        outputs=result_display,
    )

demo.launch()

c:\Users\melsa\anaconda3\Lib\site-packages\gradio\utils.py:1052: UserWarning: Expected 1 arguments for function <function <lambda> at 0x000002541D444860>, received 0.
  warnings.warn(
c:\Users\melsa\anaconda3\Lib\site-packages\gradio\utils.py:1056: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x000002541D444860>, received 0.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
